<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_Cast_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---


![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png) ||
[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

#MongoDB version

#Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [ ]:
#!pip install simple-crypt
!pip install python_string_utils
!pip install pyswisseph
#!pip install pyswisseph==2.10.2.0.dev1

In [ ]:
!wget http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1

--2022-01-08 06:34:22--  http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
Resolving www.astro.com (www.astro.com)... 192.53.104.242
Connecting to www.astro.com (www.astro.com)|192.53.104.242|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223002 (218K)
Saving to: ‘seas_18.se1’

seas_18.se1         100%[===================>] 217.78K   207KB/s    in 1.1s    

2022-01-08 06:34:24 (207 KB/s) - ‘seas_18.se1’ saved [223002/223002]

--2022-01-08 06:34:24--  http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
Resolving www.astro.com (www.astro.com)... 192.53.104.242
Connecting to www.astro.com (www.astro.com)|192.53.104.242|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1304771 (1.2M)
Saving to: ‘semo_18.se1’

semo_18.se1         100%[===================>]   1.24M   769KB/s    in 1.7s    

2022-01-08 06:34:26 (769 KB/s) - ‘semo_18.se1’ saved [1304771/1304771]

--2022-01-08 06:34:26--  http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
Resolvi

In [ ]:
!mkdir ephe
!mv *.se1 ephe

In [ ]:
import swisseph as swe
swe.version

'2.08'

In [ ]:
#Constants and Flags necessary for this application are set here
#
swe.set_ephe_path('/content/ephe') # set path to ephemeris files
# Calendar : Julian or Gregorian
SE_GREG_CAL = 1
gregflag = SE_GREG_CAL
#
# Ayanamsha type : Lahiri = 1
swe.set_sid_mode(1)  # Lahiri Aynamsha
#
# whether speed will be calculated along with position of planet
SEFLG_SPEED = int(256)
iflag = SEFLG_SPEED
#hsys = P, Placidus
#ascii P = 080
hsysP =  bytes('P', 'utf-8')

In [ ]:
# Subtracts angles (eg Ayanamsha) from Longitudes 
# Required for conversion from Tropical (Western) to Sidereal (Indian) zodiac
def p21Sub (x,y):
    retVal = round(x - y,3)
    if retVal < 0:
        return retVal + 360
    else:
        return retVal

#Personal Data


## Primary Data Input

In [ ]:
pName = 'NDModi'                   # person identifier
pBirthLocation = 'Vadnagar, India'   # Place of Birth
pInfo = 'Politics, Administrator'  # Additional Details
LBDY = 1950                         # Year of Birth, Local Time
LBDM = 9                           # Month of Birth, Local Time
LBDD = 17                           # Day of Birth, Local Time

LBTh = 9                           # Hour of Birth, Local Time, 24 Hour Clock
LBTm = 34                           # Minute of Birth, Local Time
LBTs = 0                            # Second of Birth, Local Time

tZone = 5.5                         # Time Zone Offset. East of Greenwich is +ve

pBirthDate = str(LBDY)+'-'+str(LBDM)+'-'+str(LBDD)
pBirthTime = str(LBTh)+':'+str(LBTm)

print(pName,pBirthDate,pBirthTime, pBirthLocation,tZone)

NDModi 1950-9-17 9:34 Vadnagar, India 5.5


## Data Conversions

In [ ]:
# Date & Time of Birth
#Time Zone Conversion
UTCdt = swe.utc_time_zone(LBDY,LBDM,LBDD,LBTh,LBTm,LBTs,tZone)
print(UTCdt)

(1950, 9, 17, 4, 3, 59.99999999999915)


In [ ]:
# Date & Time of Birth
# Convert UTC date to Julian Date
JD = swe.utc_to_jd(UTCdt[0],UTCdt[1],UTCdt[2],UTCdt[3],UTCdt[4],0,gregflag) # Second Value = 0
print (JD[0], JD[1])
# JD[0] is Ephemeris Time
# JD[1] is Universal Time 
natalUT = JD[1]
print(natalUT, 'Julian Date of DTofB in Universal Time')

2433541.6690880447 2433541.66875
2433541.66875 Julian Date of DTofB in Universal Time


In [ ]:
#
# Get Lat and Long of Place of Birth
# https://pypi.org/project/geopy/
#

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Parashar21")
#location = geolocator.geocode("175 5th Avenue NYC")
#location = geolocator.geocode("36B Panditiya Road, Kolkata 700029, India")
location = geolocator.geocode(pBirthLocation)
print(location.address)

print(location.latitude, location.longitude)

#print(location.raw)
natalLAT = round(location.latitude,2)
natalLON = round(location.longitude,2)
#natalLAT = 23.78
#natalLON = 72.61

print (natalLAT, natalLON)


Vadnagar, NH168K, Vadnagar, Vadnagar Taluka, Mahesana District, Gujarat, 384355, India
23.7853622 72.6332132
23.79 72.63


#Calculations

##Ayanamsha

In [ ]:
#Ayanamsha
ayanamsha = swe.get_ayanamsa(natalUT)
print('Lahiri Ayanamsha :', ayanamsha)

Lahiri Ayanamsha : 23.168596401905525


##Position of Planets

In [ ]:
#Position of Planets
#
# body 0 = Sun, 1 = Moon, 2=Merc, 3=Ven,4=Mars, 5=Jup,6=Sat,11 = True Node (Rahu)
bLon = []
bRet = []
for body in [0,1,2,3,4,5,6,11]:
    pData = swe.calc_ut(natalUT,body,iflag)
    bLon.append(pData[0][0])
    if pData[0][3] >= 0:
        bRet.append('False')
    else :
        bRet.append('True')
for ix in range(len(bLon)) : print(ix, bLon[ix], bRet[ix])

0 173.70153511513092 False
1 241.10801110249824 False
2 174.01669591038853 True
3 158.78118819325124 False
4 234.05744869607858 False
5 329.7701270221012 True
6 172.81259672693156 False
7 358.3791482368918 False


##House Position and Ascendants

In [ ]:
#House Position and Ascendants
P = swe.houses(natalUT,natalLAT,natalLON,hsysP)
#P = swe.houses_ex(natalUT,natalLAT,natalLON,hsysP)
for ix in range(len(P[0])) : print(ix, P[0][ix])
print('--')
for ix in range(len(P[1])) : print(ix, P[1][ix])

0 215.22847918934025
1 244.37764727210242
2 274.8148082930347
3 306.51592675177596
4 338.5158508028682
5 8.601719585111255
6 35.22847918934025
7 64.37764727210242
8 94.81480829303467
9 126.51592675177596
10 158.51585080286821
11 188.60171958511123
--
0 215.22847918934025
1 126.51592675177596
2 128.90482014038182
3 106.72452642971348
4 221.33751129725653
5 229.38846206839727
6 201.23094555126573
7 49.38846206839728


The output generated here matches with the output generated by the Windows program swetest

#Creating py21 file

In [ ]:
L1 = ''
L1 = L1 + 'La:'+str(p21Sub(P[1][0],ayanamsha))+','
L1 = L1 + 'Su:'+str(p21Sub(bLon[0],ayanamsha))+','
L1 = L1 + 'Mo:'+str(p21Sub(bLon[1],ayanamsha))+','
L1 = L1 + 'Ma:'+str(p21Sub(bLon[4],ayanamsha))+','
L1 = L1 + 'Me:'+str(p21Sub(bLon[2],ayanamsha))+','
L1 = L1 + 'Ju:'+str(p21Sub(bLon[5],ayanamsha))+','
L1 = L1 + 'Ve:'+str(p21Sub(bLon[3],ayanamsha))+','
L1 = L1 + 'Sa:'+str(p21Sub(bLon[6],ayanamsha))+','
L1 = L1 + 'Ra:'+str(p21Sub(bLon[7],ayanamsha))+','
L1 = L1 + 'Ke:'+str(p21Sub(p21Sub(bLon[7],ayanamsha),180))
L1 = L1 + '\n'
print(L1)

La:192.06,Su:150.533,Mo:217.939,Ma:210.889,Me:150.848,Ju:306.602,Ve:135.613,Sa:149.644,Ra:335.211,Ke:155.211



In [ ]:
L2 = ''
L2 = L2 + 'La:'+'False'+','
L2 = L2 + 'Su:'+bRet[0]+','
L2 = L2 + 'Mo:'+bRet[1]+','
L2 = L2 + 'Ma:'+bRet[4]+','
L2 = L2 + 'Me:'+bRet[2]+','
L2 = L2 + 'Ju:'+bRet[5]+','
L2 = L2 + 'Ve:'+bRet[3]+','
L2 = L2 + 'Sa:'+bRet[6]+','
L2 = L2 + 'Ra:'+'False'+','
L2 = L2 + 'Ke:'+'False'
L2 = L2 + '\n'
print(L2)

La:False,Su:False,Mo:False,Ma:False,Me:True,Ju:True,Ve:False,Sa:False,Ra:False,Ke:False



In [ ]:
L3 = pName+';'+pBirthDate+';'+pBirthTime+';'+pBirthLocation
print(L3)

NDModi;1950-9-17;9:34;Vadnagar, India


In [ ]:
writefile = pName+'.py21'
f1 = open(writefile, 'w')
f1.write(L1)  # python will convert \n to os.linesep
f1.write(L2)
f1.write(L3)
f1.close()

In [ ]:
!cat NDModi.py21

La:192.06,Su:150.533,Mo:217.939,Ma:210.889,Me:150.848,Ju:306.602,Ve:135.613,Sa:149.644,Ra:335.211,Ke:155.211
La:False,Su:False,Mo:False,Ma:False,Me:True,Ju:True,Ve:False,Sa:False,Ra:False,Ke:False
NDModi;1950-9-17;9:34;Vadnagar, India

#Store in MongoDb





In [ ]:
from pymongo import MongoClient
import pandas as pd
import dateutil
import string_utils

In [ ]:
!apt install mongodb > /dev/null
 
#Local Server Need Not be started for Remote Operations
!service mongodb start



 * Starting database mongodb
   ...done.


In [ ]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'local']

In [ ]:
# Delete a database
client.drop_database('P21DB')
# Create a new collection. We can create the collection or leave to MongoDB to create it as soon as a document is generated.
db = client.P21DB
db.create_collection('Charts')       # Optional collection creation

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'P21DB'), 'Charts')

In [ ]:
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
list (db.list_collections())
# empty list '[]' means that there are not collections in database

[{'idIndex': {'key': {'_id': 1}, 'name': '_id_', 'ns': 'P21DB.Charts', 'v': 2},
  'info': {'readOnly': False,
   'uuid': UUID('c34922bd-4024-41a4-aa33-949d8e3c49f5')},
  'name': 'Charts',
  'options': {},
  'type': 'collection'}]

In [ ]:
chart1 = {  'name' : string_utils.shuffle(pName),          # Obfuscating the name with random shuffle                       
            'DoB' : pBirthDate,                                        
            'ToB' : pBirthTime,                                         
            'LoB' : pBirthLocation,
            'info': pInfo                                            
       }
print(chart1)

{'name': 'MDNdoi', 'DoB': '1950-9-17', 'ToB': '9:34', 'LoB': 'Vadnagar, India', 'info': 'Politics, Administrator'}


In [ ]:
kollection = db.Charts                   # Set the collection to work with
insert_result = kollection.insert_one(chart1)
insert_result.acknowledged    # Confirms that insert is successful

True

In [ ]:
c = kollection.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('61d94b93741fc823a91fef48'), 'name': 'NDModi', 'DoB': '1950-9-17', 'ToB': '9:34', 'LoB': 'Vadnagar, India'}
{'_id': ObjectId('61d94bef741fc823a91fef49'), 'name': 'NDModi', 'DoB': '1950-9-17', 'ToB': '9:34', 'LoB': 'Vadnagar, India', 'info': 'Politics, Administrator'}
{'_id': ObjectId('61d94dcc741fc823a91fef4a'), 'name': 'DiNdoM', 'DoB': '1950-9-17', 'ToB': '9:34', 'LoB': 'Vadnagar, India', 'info': 'Politics, Administrator'}
{'_id': ObjectId('61d94e09741fc823a91fef4b'), 'name': 'MDNdoi', 'DoB': '1950-9-17', 'ToB': '9:34', 'LoB': 'Vadnagar, India', 'info': 'Politics, Administrator'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/"My Drive"/Prithwis/Parashar21/py21-files

DaVinci.py21  itAnch.py21  niCha01.py21  wrCal02.py21
imCal02.py21  NDModi.py21  pmCal02.py21


In [ ]:
#!cp *.py21 /content/drive/"My Drive"/Personal/Parashar21/datafiles
!cp itAnch.py21 /content/drive/"My Drive"/Prithwis/Parashar21/py21-files

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)